# Isotropic, isothermal linear elasticity UMAT Description

## Overview

UMAT for isotropic elasticity that incorporates, axisymmetric, plane and 3D stress formulations.

Isotropic elasticity formulation derived from Chapter 4, Introduction to Linear Elasticity, Gould, P. Axisymmetric and plane stress derived from EN2340 Computational Methods in Solid Mechanics, Allan Bower, Brown University. 

## Elastic Properties and Calculated Parameters

In this case the material is defined by its given Young's Modulus, $E$ and Poisson's ratio, $\nu$. The Young's modulus measures the tensile stiffness of a solid, this is achieved by quantifying the relationship between tensile stress $\sigma$ and axial strain $\varepsilon$ in the linear elastic region of a material and is given by the following formula;

\begin{equation}
E = \frac{\sigma}{\varepsilon}
\end{equation}

Poisson's ratio is a measure of material deformation in directions perpendicular to the direction of loading, given by the negative of the ratio of transverse strains, $\varepsilon_{trans}$ to axal strains, $\varepsilon_{axial}$  as shown in the following formula; 

\begin{equation}
    \nu = - \frac{\varepsilon_{trans}}{\varepsilon_{axial}}
\end{equation}

From these properties, Lamé's first parameter, $\mu$ and Lamé's second parameter, $\lambda$ are now defined; 

\begin{equation}
    \mu = \frac{E}{2+(1+\nu)}
\end{equation}

\begin{equation}
    \lambda = \frac{\nu E}{(1+\nu) (1-2 \nu)}
\end{equation}

In addition to these parameters for the plane stress case we will define the following parameter $\kappa$; 

\begin{equation}
    \kappa = \frac{E}{1-\nu^2}
\end{equation}

## 3D Case

For homogenous and isotropic materials Lamé's parameters define Hooke's law in three dimensions, where the stress tensor in index notation $\sigma_{ij}$ may be defined as;

\begin{equation}
    \sigma_{ij} = 2 \mu \varepsilon_{ij} + \lambda \delta_{ij} \varepsilon_{kk} 
\end{equation}

Where $\varepsilon_{ij}$ is the strain tensor in index notation and $\delta_{ij}$ is the the Kronecker delta function, this is simply a two variable function where the function is equal to one where the variables are equal and zero otherwise, this is expressed below; 

\begin{equation}
    \delta_{ij} = \begin{cases} 0 & \text{if} & i \neq j \\
    1 & \text{if} & i = j 
    \end{cases} 
\end{equation}

The stress tensor can then be written in the following matrix formulation;

\begin{equation}
    \begin{bmatrix}
        \sigma_{11} \\
        \sigma_{22} \\
        \sigma_{33} \\
        \sigma_{12} \\
        \sigma_{23} \\
        \sigma_{31}
    \end{bmatrix}
    = 
    \begin{bmatrix}
        2 \mu + \lambda & \lambda & \lambda & 0 & 0 & 0 \\
        \lambda & 2 \mu + \lambda & \lambda & 0 & 0 & 0 \\
        \lambda & \lambda & 2 \mu + \lambda & 0 & 0 & 0 \\
        0 & 0 & 0 & 2 \mu & 0 & 0 \\
        0 & 0 & 0 & 0 & 2 \mu & 0 \\
        0 & 0 & 0 & 0 & 0 & 2 \mu
    \end{bmatrix}
    \begin{bmatrix}
        \varepsilon_{11} \\
        \varepsilon_{22} \\
        \varepsilon_{33} \\
        \varepsilon_{12} \\
        \varepsilon_{23} \\
        \varepsilon_{31}
    \end{bmatrix}
\end{equation}

This stress strain relationship is implemented in the UMAT using the following block of code;


```fortran
DO K1 = 1,NDI
	DO K2 = 1,NDI
		DDSDDE(K2,K1) = ELAMBDA
	END DO
	DDSDDE(K1,K1) = TWO*EMU+ELAMBDA
END DO
	DO K1 = NDI+1,NTENS
        DDSDDE(K1,K1) = TWO*EMU
	END DO
```